In [2]:
pip install datasets transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00


In [3]:
!pip install accelerate>=0.20.1
import accelerate
print(accelerate.__version__)

0.25.0


In [4]:
import transformers

print(transformers.__version__)

4.35.2


In [ ]:
pip install python-docx


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
import os
from docx import Document
import re
import shutil


def preprocess_docx(docx_path, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Load the Word document
    doc = Document(docx_path)

    # Flag to determine whether to skip content
    skip_content = False

    output_text_path = os.path.join(output_folder, "extracted_text.txt")

    with open(output_text_path, "w", encoding="utf-8") as output_file:
        for i, paragraph in enumerate(doc.paragraphs):
            # Check if the starting or ending lines are present in the paragraph
            if "3rd Generation Partnership Project (3GPP TM)" in paragraph.text:
                skip_content = True
                continue
            elif "All rights reserved." in paragraph.text:
                skip_content = False
                continue

            # Skip content if the flag is True
            if skip_content:
                continue

            # Process text as it is
            text = paragraph.text + "\n"
            output_file.write(text)

def extract_and_append_chunks(text, output_file):
    # Find the indices of the first occurrence of '3GPP TS' and 'Contents'
    first_3gpp_ts = text.find('3GPP TS')
    first_contents = text.find('Contents')

    # Find the indices of the second occurrence of 'Foreword'
    first_foreword = text.find('Foreword')
    second_foreword = text.find('Foreword', first_foreword + 1)

    # Find the index of the first occurrence of 'Introduction' after the second 'Foreword'
    first_introduction = text.find('Introduction', second_foreword)

    # Find the indices of the second occurrence of '1 Scope'
    first_scope = text.find('1	Scope')
    second_scope = text.find('1	Scope', first_scope + 1)

    # Extract the first chunk between '3GPP TS' and 'Contents'
    if first_3gpp_ts != -1 and first_contents != -1:
        first_chunk = text[first_3gpp_ts:first_contents]
        with open(output_file, "a", encoding="utf-8") as file:
            file.write(first_chunk.strip())

    # Extract the second chunk between 'Contents' and the second 'Foreword'
    if first_contents != -1 and second_foreword != -1:
        second_chunk = text[first_contents:second_foreword]
        with open(output_file, "a", encoding="utf-8") as file:
            file.write(second_chunk.strip())

    # Extract the third chunk between the second 'Foreword' and the first 'Introduction'
    if second_foreword != -1 and first_introduction != -1:
        third_chunk = text[second_foreword:first_introduction]
        with open(output_file, "a", encoding="utf-8") as file:
            file.write(third_chunk.strip())

    # Extract the fourth chunk between 'Introduction' and the second '1 Scope'
    if first_introduction != -1 and second_scope != -1:
        fourth_chunk = text[first_introduction:second_scope]
        with open(output_file, "a", encoding="utf-8") as file:
            file.write(fourth_chunk.strip())

    # Extract the remaining text based on the given regex pattern after Chunk 4
    remaining_text = text[second_scope:]
    sections = re.split(r'\n(?=\d+(\.\d+){0,2}\s)(?!\.\d)', remaining_text)

    # Print non-empty and relevant sections
    chunk_count = 4
        # Append the extracted chunks to the output file
    with open(output_file, "a", encoding="utf-8") as file:
        for i, section in enumerate(sections, start=1):
            if section and section.strip() and not re.match(r'^\.?\d+\s*$', section.strip()):
                file.write(section.strip())

# Specify your Google Drive folder path containing DOCX files
input_folder = "/content/drive/MyDrive/Radcom_Fine_tuning/docx"

# Specify the output file path
output_file_path = "/content/drive/MyDrive/Radcom_Fine_tuning/docx/all_chunks.txt"

# Iterate through each file in the input folder
for filename in os.listdir(input_folder):
    if filename.endswith(".docx"):
        # Construct the full path to the DOCX file
        docx_path = os.path.join(input_folder, filename)

        # Preprocess the DOCX file and save extracted text to a temporary file
        temp_output_folder = "/content/drive/MyDrive/Radcom_Fine_tuning/temp"
        preprocess_docx(docx_path, temp_output_folder)

        # Read the preprocessed text from the temporary file
        with open(os.path.join(temp_output_folder, "extracted_text.txt"), 'r', encoding='utf-8') as temp_file:
            preprocessed_text = temp_file.read()

        # Extract and append chunks to the final output file
        extract_and_append_chunks(preprocessed_text, output_file_path)

# Remove the temporary output folder
shutil.rmtree(temp_output_folder)


In [5]:
from sklearn.model_selection import train_test_split
from datasets import load_dataset

# Read the entire text data from the file
file_path = "/content/drive/MyDrive/Radcom_Fine_tuning/docx/all_chunks.txt"
with open(file_path, 'r', encoding='utf-8') as file:
    text_data = file.read()

# Split the data into train and validation sets
train_data, validation_data = train_test_split(text_data.splitlines(), test_size=0.2, random_state=42)

# Print the first 10 lines of the train_data
for line in train_data[:10]:
    print(line)

# Save the split data to separate files
train_file_path = "/content/drive/MyDrive/Radcom_Fine_tuning/docx/train_data.txt"
validation_file_path = "/content/drive/MyDrive/Radcom_Fine_tuning/docx/validation_data.txt"

train_data_str = "\n".join(train_data)
validation_data_str = "\n".join(validation_data)

with open(train_file_path, 'w', encoding='utf-8') as train_file:
    train_file.write(train_data_str)

with open(validation_file_path, 'w', encoding='utf-8') as validation_file:
    validation_file.write(validation_data_str)

# Load the split data into datasets
datasets = load_dataset("text", data_files={"train": train_file_path, "validation": validation_file_path})

# Print a sample from the loaded datasets
print(datasets["train"][0])
print(datasets["validation"][0])


NOTE: There could be other interfaces and/or protocols requiring testing under clause 4.4.4 of TS 33.117 [3].
Table 4.2-1: Scope of NF domain that provide the LI functions
NOTE:	In case that the LMF requests broadcast keys from the Central Key Management Function, only symmetric keys can be generated, because the LMF does not know whether the requesting UE intends to broadcast or receive SLPP signaling when sending registration request.6.26.3	Evaluation
-	UE uses the public key of the CN and the cell ID specific PVT to verify the signature.
-	when the PTC server sends a TBCP Talk Burst Revoke to a PTC target.
-	The solution shall provide confidentiality and/or integrity end-to-end between source and destination network for specific message elements identified in the present document. For this requirement to be fulfilled, the SEPP – cf [2], clause 6.2.17 shall be present at the edge of the source and destination networks dedicated to handling e2e Core Network Interconnection Security. T

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

{'text': 'NOTE: There could be other interfaces and/or protocols requiring testing under clause 4.4.4 of TS 33.117 [3].'}
{'text': '-\tin general, UE and NAF will not yet share the key(s) required to protect the reference point Ua. If they already do (i.e. if a key Ks_NAF for the corresponding key derivation parameter NAF_Id is already available), the UE and the NAF can start to securely communicate right away. If the UE and the NAF do not yet share a key, and if the use of a Ks derived from an AKA-based GBA variant according to clauses\xa04.5.3, 5.5.3, or I.5.3, is not possible, the UE proceeds as follows:'}


In [6]:
model_checkpoint = "distilroberta-base"

In [7]:
from transformers import AutoTokenizer


In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map (num_proc=4):   0%|          | 0/96944 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/24237 [00:00<?, ? examples/s]

In [10]:
vocab_size = tokenizer.vocab_size
print("Vacabulory Size:", vocab_size)

vocab_mapping = tokenizer.get_vocab()
#print(vocab_mapping["'"])
# Display a few examples (change the range or use all keys for the complete mapping)
for word, token_id in list(vocab_mapping.items())[:10]:
    print(f"{word}: {token_id}")

Vacabulory Size: 50265
Got: 40084
795: 36346
Ġpristine: 27441
ĠPTSD: 24679
ĠBie: 33541
ĠIntake: 46793
Ġdoesnt: 47320
Ġgradually: 9097
Ġ1950: 9323
763: 39850


In [11]:
print(datasets["train"][1])

{'text': 'Table 4.2-1: Scope of NF domain that provide the LI functions'}


In [12]:
len(tokenized_datasets["train"][1]["input_ids"])

18

In [13]:
tokenized_datasets["train"][1]["input_ids"]

[0,
 41836,
 204,
 4,
 176,
 12,
 134,
 35,
 30108,
 9,
 33861,
 11170,
 14,
 694,
 5,
 25709,
 8047,
 2]

In [14]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 96944
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 24237
    })
})

In [15]:
# block_size = tokenizer.model_max_length
block_size = 128

In [18]:
#this function is to combine the entire tokens and divide them into block size of 128. Now after running this block the result will have two keys "train" and "validation",
#train will have three keys "input_ids", masking, and "labels". Note the input id size if of block size 128 and the token are same in input -id and label column

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    #print(concatenated_examples)
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [19]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

Map (num_proc=4):   0%|          | 0/96944 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/24237 [00:00<?, ? examples/s]

In [20]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 26341
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 6559
    })
})

In [21]:
print(lm_datasets["train"]["labels"][0])
print(lm_datasets["train"][0]["input_ids"])

[0, 46599, 35, 345, 115, 28, 97, 33285, 8, 73, 368, 18956, 7980, 3044, 223, 13166, 204, 4, 306, 4, 306, 9, 12932, 2357, 4, 21598, 646, 246, 8174, 2, 0, 41836, 204, 4, 176, 12, 134, 35, 30108, 9, 33861, 11170, 14, 694, 5, 25709, 8047, 2, 0, 46599, 35, 50117, 1121, 403, 14, 5, 31160, 597, 5034, 2308, 10654, 31, 5, 1505, 4300, 1753, 42419, 6, 129, 42662, 25286, 10654, 64, 28, 5129, 6, 142, 5, 31160, 597, 473, 45, 216, 549, 5, 14030, 37084, 9731, 7, 2308, 50, 1325, 10962, 5756, 22436, 77, 3981, 5770, 2069, 4, 401, 4, 2481, 4, 246, 50117, 717, 6486, 9762, 2, 0, 12, 50117, 9162, 2939, 5, 285, 762, 9, 5, 17313, 8, 5, 3551, 4576, 2167, 23774, 565]
[0, 46599, 35, 345, 115, 28, 97, 33285, 8, 73, 368, 18956, 7980, 3044, 223, 13166, 204, 4, 306, 4, 306, 9, 12932, 2357, 4, 21598, 646, 246, 8174, 2, 0, 41836, 204, 4, 176, 12, 134, 35, 30108, 9, 33861, 11170, 14, 694, 5, 25709, 8047, 2, 0, 46599, 35, 50117, 1121, 403, 14, 5, 31160, 597, 5034, 2308, 10654, 31, 5, 1505, 4300, 1753, 42419, 6, 129, 42662

In [22]:
# Access the training split
training_data = lm_datasets["train"]

# Retrieve the first batch of examples
first_batch = training_data[:1]

# Print the first batch
print(first_batch)


{'input_ids': [[0, 46599, 35, 345, 115, 28, 97, 33285, 8, 73, 368, 18956, 7980, 3044, 223, 13166, 204, 4, 306, 4, 306, 9, 12932, 2357, 4, 21598, 646, 246, 8174, 2, 0, 41836, 204, 4, 176, 12, 134, 35, 30108, 9, 33861, 11170, 14, 694, 5, 25709, 8047, 2, 0, 46599, 35, 50117, 1121, 403, 14, 5, 31160, 597, 5034, 2308, 10654, 31, 5, 1505, 4300, 1753, 42419, 6, 129, 42662, 25286, 10654, 64, 28, 5129, 6, 142, 5, 31160, 597, 473, 45, 216, 549, 5, 14030, 37084, 9731, 7, 2308, 50, 1325, 10962, 5756, 22436, 77, 3981, 5770, 2069, 4, 401, 4, 2481, 4, 246, 50117, 717, 6486, 9762, 2, 0, 12, 50117, 9162, 2939, 5, 285, 762, 9, 5, 17313, 8, 5, 3551, 4576, 2167, 23774, 565]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [23]:
from transformers import AutoModelForMaskedLM
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
num_layers = model.roberta.config.num_hidden_layers
print(f"Number of layers: {num_layers}")

Number of layers: 6


In [25]:
print("Model Parameters:")
for name, param in model.named_parameters():
    print(f"{name}: {param.shape}")

Model Parameters:
roberta.embeddings.word_embeddings.weight: torch.Size([50265, 768])
roberta.embeddings.position_embeddings.weight: torch.Size([514, 768])
roberta.embeddings.token_type_embeddings.weight: torch.Size([1, 768])
roberta.embeddings.LayerNorm.weight: torch.Size([768])
roberta.embeddings.LayerNorm.bias: torch.Size([768])
roberta.encoder.layer.0.attention.self.query.weight: torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.query.bias: torch.Size([768])
roberta.encoder.layer.0.attention.self.key.weight: torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.key.bias: torch.Size([768])
roberta.encoder.layer.0.attention.self.value.weight: torch.Size([768, 768])
roberta.encoder.layer.0.attention.self.value.bias: torch.Size([768])
roberta.encoder.layer.0.attention.output.dense.weight: torch.Size([768, 768])
roberta.encoder.layer.0.attention.output.dense.bias: torch.Size([768])
roberta.encoder.layer.0.attention.output.LayerNorm.weight: torch.Size([768])
roberta.e

In [26]:
from transformers import Trainer, TrainingArguments

In [27]:
for param in model.parameters():
    param.requires_grad = False

for param in model.roberta.encoder.layer[-2:].parameters():
    param.requires_grad = True

for param in model.lm_head.parameters():
    param.requires_grad = True


In [28]:
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [29]:
model_name = model_checkpoint.split("/")[-1]
training_args = TrainingArguments(
    output_dir="./content/drive/MyDrive/Radcom_Fine_tuning",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=False,
    num_train_epochs=10,
)

In [30]:
import os
print(os.getcwd())


/content


In [31]:
from transformers import AdamW, get_linear_schedule_with_warmup
from transformers import Trainer, TrainingArguments

# Assume you have already defined your model, tokenizer, data_collator, and lm_datasets

# Create a Trainer instance (before creating optimizer and scheduler)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["validation"],
    data_collator=data_collator,
)

# Create optimizer
optimizer = AdamW(model.parameters())

# Access the trainer to get the number of training steps
num_training_steps = len(trainer.get_train_dataloader()) * training_args.num_train_epochs

# Create scheduler
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Pass optimizer and scheduler as a tuple
optimizers = (optimizer, scheduler)

# Set the optimizers in the Trainer
trainer.optimizers = optimizers

# Train the model
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,2.065800,1.877429
2,1.821400,1.689461
3,1.708700,1.566387
4,1.633500,1.503266
5,1.578900,1.456459
6,1.527000,1.409263
7,1.492500,1.388413
8,1.503900,1.368986
9,1.464200,1.342897
10,1.443600,1.335619


TrainOutput(global_step=32930, training_loss=1.6595912753793558, metrics={'train_runtime': 6252.6612, 'train_samples_per_second': 42.128, 'train_steps_per_second': 5.267, 'total_flos': 8733477518599680.0, 'train_loss': 1.6595912753793558, 'epoch': 10.0})

In [32]:
# Save the model
model.save_pretrained("/content/drive/MyDrive/Radcom_Fine_tuning")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/Radcom_Fine_tuning")


('/content/drive/MyDrive/Radcom_Fine_tuning/tokenizer_config.json',
 '/content/drive/MyDrive/Radcom_Fine_tuning/special_tokens_map.json',
 '/content/drive/MyDrive/Radcom_Fine_tuning/vocab.json',
 '/content/drive/MyDrive/Radcom_Fine_tuning/merges.txt',
 '/content/drive/MyDrive/Radcom_Fine_tuning/added_tokens.json',
 '/content/drive/MyDrive/Radcom_Fine_tuning/tokenizer.json')

In [33]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 3.79


In [12]:
from transformers import AutoModel, AutoTokenizer
import torch

# Load the saved tokenizer
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Radcom_Fine_tuning")

# Load the saved model
model = AutoModel.from_pretrained("/content/drive/MyDrive/Radcom_Fine_tuning")

# Encode text to obtain input IDs and attention mask
text = "This is the 3GPP document file. I am trying to finetune a LML to generate better embeddings in order to upgrade my RAG pipeline."
inputs = tokenizer(text, return_tensors="pt")
print(inputs)
# Forward pass through the model to obtain embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extract embeddings from the output
embeddings = outputs.last_hidden_state.mean(dim=1)  # You can use other pooling strategies

# Convert embeddings to a NumPy array
embeddings_np = embeddings.numpy()

# Now 'embeddings_np' contains the embeddings for the input text
print("The length of embedding for a sentence is", embeddings_np.shape)
print(len(embeddings_np[0]))
print(embeddings_np[0])


Some weights of RobertaModel were not initialized from the model checkpoint at /content/drive/MyDrive/Radcom_Fine_tuning and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'input_ids': tensor([[    0,   713,    16,     5,   155,   534,  5756,  3780,  2870,     4,
            38,   524,   667,     7,  8746,   594,  4438,    10,   226, 10537,
             7,  5368,   357, 33183,   417,  1033,    11,   645,     7,  6662,
           127,   248,  3450,  4116,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
The length of embedding for a sentence is (1, 768)
768
[ 1.04987249e-01  9.19039324e-02  9.28723365e-02  1.04465723e-01
  1.16546839e-01  4.13675979e-02  2.25924309e-02  5.40399551e-02
  2.35773232e-02 -1.30028613e-02  9.18251127e-02 -7.78134018e-02
  3.70369405e-02  6.44721417e-03  1.55870736e-01  5.11056446e-02
  1.87618524e-01  1.33006513e-01 -5.12249134e-02  1.25357762e-01
 -9.36130527e-03 -4.36266102e-02 -5.25355041e-02 -4.97835167e-02
 -1.91548511e-01  9.75299552e-02  1.85358729e-02  2.63243113e-02
 -1.80674344e-01 -1.17396981e-01 -4.52

In [11]:
model_checkpoint = "distilroberta-base"
from transformers import AutoModel, AutoTokenizer
import torch

# Load the saved tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
# Load the saved model
model = AutoModel.from_pretrained(model_checkpoint)

# Encode text to obtain input IDs and attention mask
text = "This is the 3GPP document file. I am trying to finetune a LML to generate better embeddings in order to upgrade my RAG pipeline."
inputs = tokenizer(text, return_tensors="pt")
print(inputs)
# Forward pass through the model to obtain embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Extract embeddings from the output
embeddings = outputs.last_hidden_state.mean(dim=1)  # You can use other pooling strategies

# Convert embeddings to a NumPy array
embeddings_np = embeddings.numpy()

# Now 'embeddings_np' contains the embeddings for the input text
print("The length of embedding for a sentence is", embeddings_np.shape)
print(len(embeddings_np[0]))
print(embeddings_np[0])


{'input_ids': tensor([[    0,   713,    16,     5,   155,   534,  5756,  3780,  2870,     4,
            38,   524,   667,     7,  8746,   594,  4438,    10,   226, 10537,
             7,  5368,   357, 33183,   417,  1033,    11,   645,     7,  6662,
           127,   248,  3450,  4116,     4,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
The length of embedding for a sentence is (1, 768)
768
[ 7.84278661e-02  1.04134649e-01  1.83928628e-02  4.44539860e-02
  8.40375200e-03  1.50224388e-01 -7.42406165e-03 -1.90222822e-02
 -4.45844941e-02 -2.65256837e-02  1.84326693e-02 -7.55247697e-02
  7.22437203e-02  4.60520349e-02  1.05837516e-01  9.69827175e-04
  5.68449348e-02  3.36325914e-02  1.08523862e-02  7.18468875e-02
  2.98729278e-02  1.00160025e-01 -4.31805998e-02 -2.99778916e-02
 -1.24405302e-01  9.54528973e-02  8.34735259e-02  3.18941800e-03
 -1.71048746e-01 -1.02472015e-01 -8.75

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [ ]:
from transformers import pipeline
from transformers import AutoModelForMaskedLM
from transformers import AutoTokenizer

# Load the fine-tuned model
model_checkpoint = "/content/drive/MyDrive/Radcom_Fine_tuning"  # Specify the path to your fine-tuned model checkpoint
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True, num_proc=4, remove_columns=["text"])

# Create a pipeline for generating embeddings
embedding_pipeline = pipeline(
    "feature-extraction",
    model=model,
    tokenizer=tokenizer
)

# Example textual chunks
text_chunks = ["This is the first chunk."]

# Generate embeddings for the textual chunks
embeddings = embedding_pipeline(text_chunks)
print(len(embeddings))
# Print the embeddings
for i, embedding in enumerate(embeddings):
    print(f"Embedding for chunk {i + 1}:", embedding)


1
Embedding for chunk 1: 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import pipeline

# Load your fine-tuned model and tokenizer
model_checkpoint = "/content/drive/MyDrive/Radcom_Fine_tuning"  # Replace with the path to your fine-tuned model checkpoint
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

from transformers import AutoTokenizer

model_checkpoint1 = "distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint1, use_fast=True)


# Create a pipeline for generating embeddings
embedding_pipeline = pipeline(
    "feature-extraction",
    model=model,
    tokenizer=tokenizer
)

# Example textual chunks
text_chunks = ["This is the first chunk., Another chunk goes here. One more for good measure."]

# Generate embeddings for the textual chunks
embeddings = embedding_pipeline(text_chunks)

# Print the embeddings
for i, embedding in enumerate(embeddings):
    print(f"Embedding for chunk {i + 1}:", len(embedding[0][0]))


Embedding for chunk 1: 50265
